# Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

In [1]:
# imports
import pandas as pd
import os, time,json
! pip install tmdbsimple
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook


In [2]:
# Functions:
def get_movie_with_rating(movie_id):
#     get movie object for current id
    movie=tmdb.Movies(movie_id)
#     save .info .releases dictionaries
    info=movie.info()
    releases=movie.releases()
#     Loop through countrires in releases
    for c in releases['countries']:
#         if country abbreviation==US
        if c['iso_3166_1']=='US':
#             save a 'certification' key in info with the certification
            info['certification'] = c['certification']
    return info



def write_json(new_data, filename): # Appends a list of records (new_data) to a json file (filename)
    """ Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [3]:
# locating folder with IMBD data
FOLDER = "Desktop/Data"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'title_basics.csv.gz',
 'README.md',
 '.gitattributes',
 '.git',
 'title_akas.csv.gz',
 'title_ratings.csv.gz']

# Loading in Title Basics Data

In [4]:
basics = pd.read_csv('Desktop/Data/title_basics.csv.gz')

In [5]:
# defining lists
YEARS_TO_GET = [2000,2001] # to specify years in our outer loops
errors=[] # will append errors

In [6]:
# Outer Loop:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
        # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)

    # The code below will create the file and save an empty dictionary with just imdb_id. 
    # We will be appending to this empty dictionary throughout our calls.
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
            #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
#     Inner Loop:
        #Get index and movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1442 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1568 [00:00<?, ?it/s]

In [7]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz",
                     compression="gzip", index=False)

In [8]:
final_year_df

,imdb_id
0,0


In [9]:
print(f"- Total errors: {len(errors)}")

- Total errors: 3010


In [10]:
# Loading login credentials
with open('/Users/Oliver/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
# Displaying keys of loaded in 
login.keys()

dict_keys(['api-key'])

In [11]:
tmdb.API_KEY = login['api-key']

In [12]:
# Extracting revenue, budget, and MPAA Rating